# 프롬프트 엔지니어링(Prompt Engineering)

프롬프트 엔지니어링은 단순히 질문을 던지는 것을 넘어, 모델의 작동 원리와 '인컨텍스트 러닝(In-context Learning)' 능력을 활용해 모델의 출력을 제어하는 프로세스이다. 이는 모델의 파라미터(가중치)를 직접 수정하지 않고도 모델의 성능을 특정 태스크에 맞게 조정하는 방법론이다.

**프롬프트의 핵심 구성 요소:**

효과적인 프롬프트는 일반적으로 다음의 4가지 요소를 포함한다.

* **지시문 (Instruction):** 모델이 수행해야 할 구체적인 작업(예: 요약하라, 분류하라, 번역하라 등).
* **문맥 (Context):** 모델이 작업을 더 잘 수행하도록 돕는 배경 정보나 제약 조건.
* **입력 데이터 (Input Data):** 처리가 필요한 실제 데이터.
* **출력 지시자 (Output Indicator):** 결과물의 형식이나 스타일 지정(예: 표로 정리하라, JSON 포맷으로 출력하라 등).

**프롬프트 엔지니어링의 중요성:**

* **성능 최적화:** 같은 모델이라도 프롬프트에 따라 성능 차이가 극심하다. 잘 설계된 프롬프트는 더 작은 모델로도 큰 모델 수준의 결과를 낼 수 있게 한다.
* **비용 효율성:** 불필요한 토큰 사용을 줄이고, 파인튜닝(Fine-tuning)에 비해 적은 비용으로 도메인 특화 작업을 수행할 수 있다.
* **한계 극복:** 모델의 환각 현상을 줄이고 최신 정보를 반영(RAG와 결합 시)하도록 유도할 수 있다.


In [7]:
from openai import OpenAI
from dotenv import load_dotenv  # .env 파일의 환경변수 로드
import os                       # 환경변수 접근용
import json

load_dotenv()                   # 현재 위치의 .env를 읽어와 환경변수로 등록
api_key = os.getenv("openai_key")  # .env의 openai_key 값을 가져옴
client = OpenAI(api_key=api_key)

## 뉴스기사 제목 수정

In [16]:
response = client.chat.completions.create(    # Chat Completion API 호출
    model = 'gpt-4.1-mini',
    messages = [
        {
            "role": "system",  # 시스템 프롬프트 : 모델의 페르소나/규칙 설정
            "content": [
                {
                    "type": "text",  # 텍스트 타입 컨텐츠
                    # 모델의 역활 / 출력 예시 / 출력 규칙
                    "text": "기자들이 송고한 제목에서 맞춤법/문법/의미/어조등을 고려해 최상의 뉴스제목을 뽑아내는 20년 경력의 뉴스제목교정가이다.\n\n## Instruction\n교정이 필요한 기사 제목을 입력받아, 맞춤법과 띄어쓰기 오류, 문법 오류를 지적하고 고친 제목을 제시하세요.  \n아래 단계로 진행합니다:  \n1. 입력된 기사 제목을 면밀히 분석하여 맞춤법 오류, 띄어쓰기 실수, 문법 오류 등 문제점을 찾아 지적 항목으로 정리합니다.  \n2. 문제점을 모두 고친 교정된 기사 제목을 결과로 제시합니다.  \n3. 교정이 필요한 부분과 수정결과를 교정이유항목에 작성해주세요.\n4. 기사 제목에 오류가 여러 개 있을 경우, 각 오류를 번호를 매겨 명확히 구분하여 지적합니다.\n5. 독자의 관심을 끌수 있도록 간결하면서도 임팩트 있는 표현을 사용하세요.\n6. 어조가 지나치게 감정적이거나 부정적이라면, 적절히 중립적 표현을 사용하세요.\n7. 비속어/욕설등이 포함되어 있다면 이를 제거하고, 의미가 전달될수 있는 적절한 표현으로 수정하세요. \n\n## Output Format\n- 원래제목: [송고한 기사제목]\n- 교정제목: [교정한 기사제목]\n- 교정 이유:\n  1. [교정한 부분과 이유]\n  2. [교정한 부분과 이유]\n\n## Examples\n<예시1>  \n입력: \"코로나19 백신접종율 높히기 위한 대안마련 필요하다\"  \n출력:  \n- 원래제목: [송고한 기사제목]\n- 교정제목: \"코로나19 백신 접종률 높이기 위한 대안 마련 시급\"\n- 교정 이유:\n   1. '접종율'은 '접종률'이 맞는 표기입니다.\n   2. '높히기'는 '높이기'로, 맞춤법 오류입니다.\n   3. '대안마련'은 붙여쓰지 않고 '대안 마련'으로 띄어 써야 맞습니다.\n   4. 간결한 어미수정\n\n<예시2>  \n입력: \"정부, 연금개혁 지방화 시대 맞춰 적극 나서야한다\"  \n출력:  \n- 원래제목: [송고한 기사제목]\n- 교정제목: \"정부, 연금개혁 지방화 시대 맞춰 적극 나서야\"\n- 교정 이유:\n  - 간결한 어미 수정\n" 
                }
            ]
        },
        {
            "role": "user",    # 유저 프롬프트 : 실제 입력 데이터
            "content": [
                {
                    "type": "text",
                    "text": "## Input Data\n입력: 두쫀쿠 열풍 뭐가 존맛탱이라는 건지 알수가 없네"
                }
            ]
        }
    ],
    response_format = {"type": "text"},  # 응답 형식
    temperature = 1,                     # 모델의 랜덤성/다양성 (낮으면 일관성, 높으면 창의적)
    max_completion_tokens = 2048,        # 최대 출력 토큰
    top_p = 1,                           # 누적확률 p까지의 후보를 샘플링 (1이면 전체사용)
    frequency_penalty = 0,               # 동일 단어/구절 반복시 감점 (반복 억제)
    presence_penalty = 0,                # 이미 등장한 주제/단어 사용시 감점 (반복 억제)
    store = False                        # 응답을 서버에 저장/로깅 할지 설정 (False면 저장 안함)
)

response

ChatCompletion(id='chatcmpl-D3YdqV8n08WzFKk995wpZKbRdyFNZ', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='- 원래제목: [두쫀쿠 열풍 뭐가 존맛탱이라는 건지 알수가 없네]\n- 교정제목: "두쫀쿠 열풍, 무엇이 특별한지 알기 어렵다"\n- 교정 이유:\n  1. \'존맛탱\'은 비속어로, 공식 기사 제목에는 적절하지 않아 \'특별한\'으로 수정했습니다.\n  2. \'뭐가\'는 비격식 표현이므로 \'무엇이\'로 수정해 중립적인 어조를 유지했습니다.\n  3. \'알수가 없네\'는 구어체 표현으로, \'알기 어렵다\'로 바꾸어 적절한 문어체 표현으로 다듬었습니다.\n  4. 문장의 가독성을 높이고 명확한 의미 전달을 위해 쉼표를 추가하였습니다.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1769740518, model='gpt-4.1-mini-2025-04-14', object='chat.completion', service_tier='default', system_fingerprint='fp_e01c6f58e1', usage=CompletionUsage(completion_tokens=190, prompt_tokens=645, total_tokens=835, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached

In [17]:
response.choices[0].message.content

'- 원래제목: [두쫀쿠 열풍 뭐가 존맛탱이라는 건지 알수가 없네]\n- 교정제목: "두쫀쿠 열풍, 무엇이 특별한지 알기 어렵다"\n- 교정 이유:\n  1. \'존맛탱\'은 비속어로, 공식 기사 제목에는 적절하지 않아 \'특별한\'으로 수정했습니다.\n  2. \'뭐가\'는 비격식 표현이므로 \'무엇이\'로 수정해 중립적인 어조를 유지했습니다.\n  3. \'알수가 없네\'는 구어체 표현으로, \'알기 어렵다\'로 바꾸어 적절한 문어체 표현으로 다듬었습니다.\n  4. 문장의 가독성을 높이고 명확한 의미 전달을 위해 쉼표를 추가하였습니다.'

In [21]:
# 기사 제목 교정 API를 호출하고 결과 텍스트만 반환하는 함수
def correct_headline(headline, /, *, model='gpt-4.1-mini', temperature=1, top_p=1, max_completion_tokens=2048):

    response = client.chat.completions.create(    # Chat Completion API 호출
        model = model,
        messages = [
            {
                "role": "system",  # 시스템 프롬프트 : 모델의 페르소나/규칙 설정
                "content": [
                    {
                        "type": "text",  # 텍스트 타입 컨텐츠
                        # 모델의 역활 / 출력 예시 / 출력 규칙
                        "text": "기자들이 송고한 제목에서 맞춤법/문법/의미/어조등을 고려해 최상의 뉴스제목을 뽑아내는 20년 경력의 뉴스제목교정가이다.\n\n## Instruction\n교정이 필요한 기사 제목을 입력받아, 맞춤법과 띄어쓰기 오류, 문법 오류를 지적하고 고친 제목을 제시하세요.  \n아래 단계로 진행합니다:  \n1. 입력된 기사 제목을 면밀히 분석하여 맞춤법 오류, 띄어쓰기 실수, 문법 오류 등 문제점을 찾아 지적 항목으로 정리합니다.  \n2. 문제점을 모두 고친 교정된 기사 제목을 결과로 제시합니다.  \n3. 교정이 필요한 부분과 수정결과를 교정이유항목에 작성해주세요.\n4. 기사 제목에 오류가 여러 개 있을 경우, 각 오류를 번호를 매겨 명확히 구분하여 지적합니다.\n5. 독자의 관심을 끌수 있도록 간결하면서도 임팩트 있는 표현을 사용하세요.\n6. 어조가 지나치게 감정적이거나 부정적이라면, 적절히 중립적 표현을 사용하세요.\n7. 비속어/욕설등이 포함되어 있다면 이를 제거하고, 의미가 전달될수 있는 적절한 표현으로 수정하세요. \n\n## Output Format\n- 원래제목: [송고한 기사제목]\n- 교정제목: [교정한 기사제목]\n- 교정 이유:\n  1. [교정한 부분과 이유]\n  2. [교정한 부분과 이유]\n\n## Examples\n<예시1>  \n입력: \"코로나19 백신접종율 높히기 위한 대안마련 필요하다\"  \n출력:  \n- 원래제목: [송고한 기사제목]\n- 교정제목: \"코로나19 백신 접종률 높이기 위한 대안 마련 시급\"\n- 교정 이유:\n   1. '접종율'은 '접종률'이 맞는 표기입니다.\n   2. '높히기'는 '높이기'로, 맞춤법 오류입니다.\n   3. '대안마련'은 붙여쓰지 않고 '대안 마련'으로 띄어 써야 맞습니다.\n   4. 간결한 어미수정\n\n<예시2>  \n입력: \"정부, 연금개혁 지방화 시대 맞춰 적극 나서야한다\"  \n출력:  \n- 원래제목: [송고한 기사제목]\n- 교정제목: \"정부, 연금개혁 지방화 시대 맞춰 적극 나서야\"\n- 교정 이유:\n  - 간결한 어미 수정\n" 
                    }
                ]
            },
            {
                "role": "user",    # 유저 프롬프트 : 실제 입력 데이터
                "content": [
                    {
                        "type": "text",
                        "text": f"## Input Data\n입력: {headline}"
                    }
                ]
            }
        ],
        response_format = {"type": "text"},  # 응답 형식
        temperature = temperature,                      # 모델의 랜덤성/다양성 (낮으면 일관성, 높으면 창의적)
        max_completion_tokens = max_completion_tokens,  # 최대 출력 토큰
        top_p = top_p,                                  # 누적확률 p까지의 후보를 샘플링 (1이면 전체사용)
        frequency_penalty = 0,               # 동일 단어/구절 반복시 감점 (반복 억제)
        presence_penalty = 0,                # 이미 등장한 주제/단어 사용시 감점 (반복 억제)
        store = False                        # 응답을 서버에 저장/로깅 할지 설정 (False면 저장 안함)
    )

    return response.choices[0].message.content

/ (positional-only-expression) : /의 왼쪽 매개변수는 위치인자방식으로만 호출 강제화 (headline이 첫번째 위치에서 필수입력이 강제된다.)  
\* (keyword-only-expression) : *의 오른쪽 매개변수는 키워드인자방식으로만 호출 강제화 (model=..., temperature=..., 이런식으로 키워드로만 받는다.)

In [22]:
correct_headline('두쫀쿠 열풍 뭐가 존맛탱이라는 건지 알수가 없네')

'- 원래제목: [두쫀쿠 열풍 뭐가 존맛탱이라는 건지 알수가 없네]  \n- 교정제목: "두쫀쿠 열풍, 왜 인기가 있는지 알기 어렵다"  \n- 교정 이유:  \n  1. \'두쫀쿠\'는 고유명사로써 띄어쓰기는 문제없으나, 독자의 이해를 위해 그대로 둠.  \n  2. \'존맛탱\'은 속어이자 비속어에 가까운 표현으로, 뉴스 제목에 적합하지 않아 중립적이고 공식적인 표현 \'인기\'로 수정.  \n  3. \'알수가 없네\'는 구어체이며, 맞춤법상 \'알 수가\' 띄어쓰기가 필요하고, 뉴스 제목에는 다소 감정적인 어조이므로 \'알기 어렵다\'로 중립적이고 객관적인 표현으로 변경.  \n  4. \'뭐가\'는 비격식 표현으로 \'왜\'로 수정하여 공식적이고 명확한 제목으로 개선.  \n  5. 쉼표 추가로 문장 구조 명확화.'

In [23]:
correct_headline('두쫀쿠 열풍 뭐가 존맛탱이라는 건지 알수가 없네', model='gpt-5.2')

'- 원래제목: 두쫀쿠 열풍 뭐가 존맛탱이라는 건지 알수가 없네  \n- 교정제목: "‘두쫀쿠’ 열풍, 뭐가 그렇게 맛있다는 걸까"  \n- 교정 이유:  \n  1. ‘알수가’ → ‘알 수가’: 의존 명사 ‘수’는 앞말과 띄어 써야 합니다.  \n  2. ‘존맛탱’ 삭제/순화: 비속어 표현을 중립적이고 기사에 적합한 표현(‘그렇게 맛있다’)으로 바꿨습니다.  \n  3. ‘뭐가 … 알 수가 없네’ → ‘뭐가 … 걸까’: 구어체·감정적 어조(‘없네’)를 줄이고, 독자 관심을 끄는 의문형으로 정리했습니다.  \n  4. 제목 구성 정리: 핵심 키워드(‘두쫀쿠’·‘열풍’)를 앞에 두고, 뒤에 의문을 붙여 간결성과 임팩트를 강화했습니다.'

In [27]:
headlines = [
    "주말 성수동거리에는 커플들이 그득그득 하더라",
    "졸라 빡센 작업으로 끼니 거르기 일쑤인 노동자들의 애환",
    "유투버들의 생존일기, 쉽지 않은 그들의 여정에 함께해요~"
]

for headline in headlines:
    output = correct_headline(headline)
    print(output)
    print()

- 원래제목: 주말 성수동거리에는 커플들이 그득그득 하더라  
- 교정제목: 주말 성수동 거리에는 커플들이 가득했다  
- 교정 이유:  
  1. '성수동거리'는 '성수동 거리'로 띄어 써야 합니다.  
  2. '커플들이'는 자연스러우며 수정하지 않았습니다.  
  3. '그득그득'은 잘못된 표현으로, '가득'이 맞는 표준어입니다.  
  4. '하더라'는 구어체 표현으로 뉴스 기사 제목에는 적합하지 않아, 객관적인 과거형 서술인 '가득했다'로 수정했습니다.  
  5. 전체적으로 간결하고 공식적인 뉴스 제목 어조로 다듬어 독자의 관심을 끌 수 있도록 했습니다.

- 원래제목: 졸라 빡센 작업으로 끼니 거르기 일쑤인 노동자들의 애환  
- 교정제목: 매우 고된 작업으로 끼니 거르기 일쑤인 노동자들의 고충  
- 교정 이유:  
  1. '졸라'와 '빡센'은 비속어이므로, 공식적인 뉴스 제목에 적합하지 않아 '매우 고된'으로 수정했습니다.  
  2. '애환'보다 '고충'이 뉴스 제목으로 더 명확하고 중립적인 표현입니다.  
  3. 띄어쓰기와 문법에는 문제 없으나, 어조 및 비속어 사용을 중심으로 교정하였습니다.

- 원래제목: [유투버들의 생존일기, 쉽지 않은 그들의 여정에 함께해요~]  
- 교정제목: "유튜버들의 생존 일기, 쉽지 않은 그들의 여정에 함께하세요"  
- 교정 이유:  
  1. '유투버들'은 '유튜버들'이 올바른 표기입니다.  
  2. '생존일기'는 '생존 일기'로 띄어 쓰는 것이 적절합니다.  
  3. 문장 끝의 '~' 기호는 뉴스 제목에는 부적절하므로 제거하고 정중한 권유 표현인 '함께하세요'로 바꿨습니다.  
  4. 독자의 관심을 끌면서도 깔끔하고 전문적인 뉴스 제목 톤으로 조정했습니다.



In [32]:
import json

response = client.chat.completions.create(    # Chat Completion API 호출
        model = 'gpt-4.1-mini',
        messages = [
            {
                "role": "system",  # 시스템 프롬프트 : 모델의 페르소나/규칙 설정
                "content": [
                    {
                        "type": "text",  # 텍스트 타입 컨텐츠
                        # JSON 출력 포맷/예시를 포함한 시스템 프롬프트
                        "text": "기자들이 송고한 제목에서 맞춤법/문법/의미/어조등을 고려해 최상의 뉴스제목을 뽑아내는 20년 경력의 뉴스제목교정가이다.\n\n## Instruction\n교정이 필요한 기사 제목을 입력받아, 맞춤법과 띄어쓰기 오류, 문법 오류를 지적하고 고친 제목을 제시하세요.  \n아래 단계로 진행합니다:  \n1. 입력된 기사 제목을 면밀히 분석하여 맞춤법 오류, 띄어쓰기 실수, 문법 오류 등 문제점을 찾아 지적 항목으로 정리합니다.  \n2. 문제점을 모두 고친 교정된 기사 제목을 결과로 제시합니다.  \n3. 교정이 필요한 부분과 수정결과를 교정이유항목에 작성해주세요.\n4. 기사 제목에 오류가 여러 개 있을 경우, 각 오류를 번호를 매겨 명확히 구분하여 지적합니다.\n5. 독자의 관심을 끌수 있도록 간결하면서도 임팩트 있는 표현을 사용하세요.\n6. 어조가 지나치게 감정적이거나 부정적이라면, 적절히 중립적 표현을 사용하세요.\n7. 비속어/욕설등이 포함되어 있다면 이를 제거하고, 의미가 전달될수 있는 적절한 표현으로 수정하세요. \n\n## Output Format\n**반드시 json 객체 형식을 준수하세요.**\n\n{{\n  \"original_headline\": <송고한 기사제목>,\n  \"corrected_headline\": <교정한 기사제목>,\n  \"reasons\": [\n     <교정한 부분과 이유>,\n     <교정한 부분과 이유>,\n  ] \n}}\n\n## Examples\n<예시1>  \n입력: \"코로나19 백신접종율 높히기 위한 대안마련 필요하다\"  \n출력:  \n{{\n  \"original_headline\": \"코로나19 백신접종율 높히기 위한 대안마련 필요하다\",\n  \"corrected_headline\": \"코로나19 백신 접종률 높이기 위한 대안 마련 시급\",\n  \"reasons\": [\n    \"'접종율'은 표준어가 아니며 '접종률'이 올바른 표기이다\",\n    \"'높히기'는 맞춤법 오류로 '높이기'로 수정해야 한다\",\n    \"'대안마련'은 띄어 써야 하므로 '대안 마련'으로 수정하였다\",\n    \"기사 제목에 맞게 어미를 간결하게 다듬었다\"\n  ]\n}}\n\n<예시2>  \n입력: \"정부, 연금개혁 지방화 시대 맞춰 적극 나서야한다\"  \n출력:  \n{{\n  \"original_headline\": \"정부, 연금개혁 지방화 시대 맞춰 적극 나서야한다\",\n  \"corrected_headline\": \"정부, 연금개혁 지방화 시대 맞춰 적극 나서야\",\n  \"reasons\": [\n    \"기사 제목의 문체에 맞게 불필요한 어미를 제거해 간결하게 수정하였다\"\n  ]\n}}\n" 
                    }
                ]
            },
            {
                "role": "user",    # 유저 프롬프트 : 실제 입력 데이터
                "content": [
                    {
                        "type": "text",
                        "text": f"## Input Data\n입력: 두쫀쿠 열풍 뭐가 존맛탱이라는 건지 알수가 없네"
                    }
                ]
            }
        ],
        response_format = {"type": "json_object"},      # 응답 형식을 JSON 객체로 강제
        temperature = 1,                     # 모델의 랜덤성/다양성 (낮으면 일관성, 높으면 창의적)
        max_completion_tokens = 2048,        # 최대 출력 토큰
        top_p = 1,                           # 누적확률 p까지의 후보를 샘플링 (1이면 전체사용)
        frequency_penalty = 0,               # 동일 단어/구절 반복시 감점 (반복 억제)
        presence_penalty = 0,                # 이미 등장한 주제/단어 사용시 감점 (반복 억제)
        store = False                        # 응답을 서버에 저장/로깅 할지 설정 (False면 저장 안함)
    )

json.loads(response.choices[0].message.content)

{'original_headline': '두쫀쿠 열풍 뭐가 존맛탱이라는 건지 알수가 없네',
 'corrected_headline': '두쫀쿠 열풍, 왜 인기인지 알기 어렵다',
 'reasons': ["비속어 '존맛탱'은 공식 기사 제목에 부적절하므로 '인기'로 순화하였다",
  "'알수가 없네'는 비격식적 표현으로, 공식 제목에서는 '알기 어렵다'로 수정하여 중립적이고 정중한 어조로 변경하였다",
  "'뭐가'와 '이라는 건지'는 중복된 의미를 포함해 문장이 어색하므로 '왜 인기인지'로 간결하게 다듬었다",
  '쉼표를 추가해 제목이 명확하게 구분되도록 하였다']}

In [ ]:
import json

# 기사 제목 교정 API를 호출하고 결과 JSON(dict) 형태로 반환하는 함수
def correct_headline_json(headline, /, *, model='gpt-4.1-mini', temperature=1, top_p=1, max_completion_tokens=2048):

    response = client.chat.completions.create(    # Chat Completion API 호출
        model = model,
        messages = [
            {
                "role": "system",  # 시스템 프롬프트 : 모델의 페르소나/규칙 설정
                "content": [
                    {
                        "type": "text",  # 텍스트 타입 컨텐츠
                        # JSON 출력 포맷/예시를 포함한 시스템 프롬프트
                        "text": "기자들이 송고한 제목에서 맞춤법/문법/의미/어조등을 고려해 최상의 뉴스제목을 뽑아내는 20년 경력의 뉴스제목교정가이다.\n\n## Instruction\n교정이 필요한 기사 제목을 입력받아, 맞춤법과 띄어쓰기 오류, 문법 오류를 지적하고 고친 제목을 제시하세요.  \n아래 단계로 진행합니다:  \n1. 입력된 기사 제목을 면밀히 분석하여 맞춤법 오류, 띄어쓰기 실수, 문법 오류 등 문제점을 찾아 지적 항목으로 정리합니다.  \n2. 문제점을 모두 고친 교정된 기사 제목을 결과로 제시합니다.  \n3. 교정이 필요한 부분과 수정결과를 교정이유항목에 작성해주세요.\n4. 기사 제목에 오류가 여러 개 있을 경우, 각 오류를 번호를 매겨 명확히 구분하여 지적합니다.\n5. 독자의 관심을 끌수 있도록 간결하면서도 임팩트 있는 표현을 사용하세요.\n6. 어조가 지나치게 감정적이거나 부정적이라면, 적절히 중립적 표현을 사용하세요.\n7. 비속어/욕설등이 포함되어 있다면 이를 제거하고, 의미가 전달될수 있는 적절한 표현으로 수정하세요. \n\n## Output Format\n**반드시 json 객체 형식을 준수하세요.**\n\n{{\n  \"original_headline\": <송고한 기사제목>,\n  \"corrected_headline\": <교정한 기사제목>,\n  \"reasons\": [\n     <교정한 부분과 이유>,\n     <교정한 부분과 이유>,\n  ] \n}}\n\n## Examples\n<예시1>  \n입력: \"코로나19 백신접종율 높히기 위한 대안마련 필요하다\"  \n출력:  \n{{\n  \"original_headline\": \"코로나19 백신접종율 높히기 위한 대안마련 필요하다\",\n  \"corrected_headline\": \"코로나19 백신 접종률 높이기 위한 대안 마련 시급\",\n  \"reasons\": [\n    \"'접종율'은 표준어가 아니며 '접종률'이 올바른 표기이다\",\n    \"'높히기'는 맞춤법 오류로 '높이기'로 수정해야 한다\",\n    \"'대안마련'은 띄어 써야 하므로 '대안 마련'으로 수정하였다\",\n    \"기사 제목에 맞게 어미를 간결하게 다듬었다\"\n  ]\n}}\n\n<예시2>  \n입력: \"정부, 연금개혁 지방화 시대 맞춰 적극 나서야한다\"  \n출력:  \n{{\n  \"original_headline\": \"정부, 연금개혁 지방화 시대 맞춰 적극 나서야한다\",\n  \"corrected_headline\": \"정부, 연금개혁 지방화 시대 맞춰 적극 나서야\",\n  \"reasons\": [\n    \"기사 제목의 문체에 맞게 불필요한 어미를 제거해 간결하게 수정하였다\"\n  ]\n}}\n" 
                    }
                ]
            },
            {
                "role": "user",    # 유저 프롬프트 : 실제 입력 데이터
                "content": [
                    {
                        "type": "text",
                        "text": f"## Input Data\n입력: {headline}"
                    }
                ]
            }
        ],
        response_format = {"type": "json_object"},      # 응답 형식을 JSON 객체로 강제
        temperature = temperature,                      # 모델의 랜덤성/다양성 (낮으면 일관성, 높으면 창의적)
        max_completion_tokens = max_completion_tokens,  # 최대 출력 토큰
        top_p = top_p,                                  # 누적확률 p까지의 후보를 샘플링 (1이면 전체사용)
        frequency_penalty = 0,               # 동일 단어/구절 반복시 감점 (반복 억제)
        presence_penalty = 0,                # 이미 등장한 주제/단어 사용시 감점 (반복 억제)
        store = False                        # 응답을 서버에 저장/로깅 할지 설정 (False면 저장 안함)
    )

    return json.loads(response.choices[0].message.content)  # JSON 문자열 -> 파이썬 dict로 파싱

In [34]:
# JSON(dict) 형태로 교정 결과 받기
output = correct_headline_json('두쫀쿠 열풍 뭐가 존맛탱이라는 건지 알수가 없네')
print(output)

{'original_headline': '두쫀쿠 열풍 뭐가 존맛탱이라는 건지 알수가 없네', 'corrected_headline': '두쫀쿠 열풍, 왜 인기가 있는지 알기 어렵다', 'reasons': ["비속어 '존맛탱'을 공식적이고 중립적인 표현인 '인기가 있는지'로 수정하였다", "'알수가'는 띄어쓰기 오류로 '알 수가'가 맞으나, 더 자연스러운 표현인 '알기 어렵다'로 변경하였다", '문장을 간결하고 공식적인 뉴스 제목 스타일에 맞게 다듬었다']}


In [35]:
corrected_headline = output['corrected_headline']
corrected_headline

'두쫀쿠 열풍, 왜 인기가 있는지 알기 어렵다'

In [36]:
headlines = [
    "주말 성수동거리에는 커플들이 그득그득 하더라",
    "졸라 빡센 작업으로 끼니 거르기 일쑤인 노동자들의 애환",
    "유투버들의 생존일기, 쉽지 않은 그들의 여정에 함께해요~"
]

# 리스트 컴프리헨션으로 일괄 함수 적용 후 교정된 제목만 뽑아냄
headlines = [correct_headline_json(headline)['corrected_headline'] for headline in headlines]

for headline in headlines:
    print(headline)

주말 성수동 거리에는 커플들이 가득했다
혹독한 작업으로 끼니 거르기 일쑤인 노동자들의 애환
유튜버들의 생존 일기, 쉽지 않은 여정에 함께해요


# 냉털마스터

In [8]:
# 입력 재료만으로 가능한 음식 2가지를 JSON(dict) 형태로 추천해주는 함수
def refrig_master_json(user_input: str, /, *, model='gpt-4.1-mini', temperature=1, top_p=1, max_completion_tokens=2048):

    response = client.chat.completions.create(    # Chat Completion API 호출
        model = model,
        messages = [
            {
                "role": "system",  # 시스템 프롬프트 : 모델의 페르소나/규칙 설정
                "content": [
                    {
                        "type": "text",  # 텍스트 타입 컨텐츠
                        # 시스템 프롬프트 : 재료 제한 + JSON 출력 강제 + 예시를 포함
                        "text": "# Instruction\n사용자가 입력한 냉장고 내 재료 목록만을 사용하여 만들 수 있는 음식 2가지를 추천하세요.  \n반드시 입력된 재료만 활용하며, 기본양념(간장, 소금, 설탕, 설탕, 식초, 후추 등)은 언제든 사용할 수 있다고 가정하세요.  \n입력된 목록에 없는 재료(양념 제외)는 절대 사용하지 말고, 추가 재료 없이 조리 가능한 음식만 선정하십시오.  \n음식 종류는 반드시 서로 비슷하지 않은 2가지여야 하며, 각 음식에 대한 자세한 요리 레시피(조리 순서)를 단계별로 포함하세요.\n\n- 먼저, 입력 재료로 만들 수 있는 음식 종류를 논리적으로 검토한 뒤, 각 음식이 왜 가능한지 간단히 설명해 주세요.\n- reasoning(논리 및 설명)과 conclusion(최종 추천 결과)은 반드시 JSON의 개별 필드로 구분하여 제공하십시오.\n- reasoning이 반드시 먼저, conclusion이 반드시 마지막에 위치해야 합니다.\n- 결론(conclusion)에는 각 음식명과 단계별 레시피를 포함하세요.\n- 반드시 모든 답변을 한글로 작성하세요.\n\n# Steps\n\n1. 입력 재료만 활용 가능한 음식 2가지를 선정하고, 서로 비슷하지 않은지 확인하세요.\n2. reasoning(논리/설명) 필드에: \n    - 해당 재료로 어떤 음식이 가능한지, 그 이유를 간단히 단계별 논리로 설명하세요.\n3. conclusion(최종 추천) 필드에:\n    - 각 음식의 음식명\n    - 해당 음식의 구체적 요리 레시피(순서대로 단계를 나열, 최소 3단계 이상)\n    - 위 구조로 2가지만 반드시 작성하세요.\n\n# Output Format\n\n모든 답변은 아래 JSON 구조로 출력하세요.  \n- \"reasoning\": 각 음식이 왜 가능한지 단계별 논리와 검토(한글 서술, 리스트)\n- \"conclusion\": 음식명과 상세한 단계별 레시피(한글 서술, 리스트. 각 요소는 {\"food_name\": \"음식명\", \"recipe: [\"레시피1\", \"레시피2\"]} 형식)\n\n# Examples\n\n사용자 입력 예시:\n- 입력: 계란, 양파, 당근\n\n출력 예시(JSON):\n\n{\n  \"reasoning\": [\n    \"계란, 양파, 당근만 사용하여 만들 수 있는 요리를 검토합니다.\",\n    \"계란과 채소(양파, 당근)만으로 달걀전이 가능합니다. 채소를 잘게 썰어 계란과 섞어 부치면 완성할 수 있습니다.\",\n    \"계란찜 역시 이 재료로 만들 수 있습니다. 계란을 풀고 다진 채소를 섞은 후, 찜기를 사용해 익히면 완성됩니다.\"\n  ],\n  \"conclusion\": [\n    {\n      \"food_name\": \"달걀전\",\n      \"recipe\": [\n        \"1. 양파와 당근을 잘게 썰어줍니다.\",\n        \"2. 계란을 풀고 썰어둔 양파와 당근, 소금, 후추를 넣고 섞습니다.\",\n        \"3. 달궈진 팬에 기름을 두르고 반죽을 얇게 올린 후, 앞뒤로 노릇하게 부칩니다.\"\n      ]\n    },\n    {\n      \"food_name\": \"계란찜\",\n      \"recipe\": [\n        \"1. 계란을 볼에 넣고 곱게 풀어줍니다.\",\n        \"2. 다진 양파와 당근, 소금, 후추를 계란물에 넣고 섞습니다.\",\n        \"3. 뚝배기나 내열 용기에 재료를 옮겨 담고, 중탕 또는 전자레인지로 익혀 부드럽게 완성합니다.\"\n      ]\n    }\n  ]\n}\n\n(실제 예시는 입력 재료와 음식에 따라 달라지며, 각 음식의 레시피 단계는 3단계 이상, 충분히 구체적으로 작성하십시오.)\n\n# Notes\n\n- 반드시 입력 재료만 사용하고, 음식명 및 조리법 전부 한글로 기입하세요.\n- 각 추천 요리는 서로 다른 종류여야 하며, 각 음식마다 레시피 단계는 구체적이고 논리적으로 작성돼야 합니다.\n- reasoning(논리/설명) → conclusion(최종 추천 및 레시피) 순서는 꼭 지켜야 합니다.\n- 답변 형식은 반드시 JSON이어야 하며, 한글로만 작성하세요.\n\n[중요: 2가지 음식 추천, 상세 단계별 한글 레시피, 입력 재료만 허용, 양념장은 보유 가정, 항상 reasoning이 먼저, conclusion이 뒤, 반드시 JSON, 예시 구조 참고, 모든 답변은 한글로!]" 
                    }
                ]
            },
            {
                "role": "user",    # 유저 프롬프트 : 실제 입력 데이터
                "content": [
                    {
                        "type": "text",
                        "text": user_input
                    }
                ]
            }
        ],
        response_format = {"type": "json_object"},      # 응답 형식을 JSON 객체로 강제
        temperature = temperature,                      # 모델의 랜덤성/다양성 (낮으면 일관성, 높으면 창의적)
        max_completion_tokens = max_completion_tokens,  # 최대 출력 토큰
        top_p = top_p,                                  # 누적확률 p까지의 후보를 샘플링 (1이면 전체사용)
        frequency_penalty = 0,               # 동일 단어/구절 반복시 감점 (반복 억제)
        presence_penalty = 0,                # 이미 등장한 주제/단어 사용시 감점 (반복 억제)
        store = False                        # 응답을 서버에 저장/로깅 할지 설정 (False면 저장 안함)
    )

    return json.loads(response.choices[0].message.content)  # JSON 문자열 -> 파이썬 dict로 파싱

In [9]:
refrig_master_json("계란, 양파, 당근")

{'reasoning': ['계란, 양파, 당근만으로 만들 수 있는 요리를 검토합니다.',
  "첫 번째로, 계란과 채소(양파, 당근)를 섞어 부치는 '달걀전'이 가능합니다. 부침개와 비슷한 방식으로, 재료를 잘게 썰어 계란과 섞고 팬에 부치면 됩니다.",
  '두 번째로, 계란찜도 가능합니다. 계란을 풀고 다진 양파와 당근을 넣어 소금과 후추로 간을 한 뒤 따뜻하게 익히면 부드러운 계란찜이 완성됩니다.',
  "두 음식은 조리방법이 서로 다르며, '달걀전'은 튀기듯이 부치는 요리, '계란찜'은 찜으로 익히는 요리로 서로 비슷하지 않으므로 적합합니다."],
 'conclusion': [{'food_name': '달걀전',
   'recipe': ['1. 양파와 당근을 껍질을 벗기고 깨끗이 씻어 곱게 다져줍니다.',
    '2. 볼에 계란을 넣고 소금과 후추로 간을 한 뒤 잘 풀어줍니다.',
    '3. 계란물에 다진 양파와 당근을 넣고 고루 섞어줍니다.',
    '4. 팬을 중불로 달군 뒤 식용유를 두르고 반죽을 한 숟가락씩 떠서 납작하게 펴서 앞뒤로 노릇하게 부칩니다.',
    '5. 완성된 달걀전을 접시에 담아 냅니다.']},
  {'food_name': '계란찜',
   'recipe': ['1. 계란을 깨뜨려 볼에 넣고 잘 저어 풀어줍니다.',
    '2. 양파와 당근을 다져 계란물에 넣은 뒤 소금과 후추로 간을 맞춥니다.',
    '3. 내열 용기에 계란물을 붓고 랩을 덮거나 뚜껑을 덮습니다.',
    '4. 중탕 또는 전자레인지에 넣어 5~7분간 부드럽게 익힙니다.',
    '5. 익었는지 확인하고 식혀서 접시에 담아 서빙합니다.']}]}

In [10]:
output = refrig_master_json("계란, 찬밥, 파김치, 장조림")

for food in output['conclusion']:             # 추천 결과(conclusion) 음식 2개를 순회
    print(f'추천음식 : {food["food_name"]}')   # 음식명 출력
    print('레시피 : ')
    for step in food['recipe']:              # 해당 음식의 레시피 단계들을 순회
        print(' ', step)                     # 단계별 레시피를 들여쓰기 하여 출력
    print()

추천음식 : 계란 볶음밥
레시피 : 
  1. 계란 2개를 깨서 그릇에 풀어둡니다.
  2. 팬에 기름을 두르고 중불로 달군 후, 풀어둔 계란을 넣어 반숙 형태로 살짝 익혀줍니다.
  3. 찬밥 한 그릇 분량을 넣고 계란과 잘 섞이도록 볶습니다.
  4. 소금과 후추로 간을 맞추고, 볶으면서 밥알이 골고루 퍼지도록 약 3~5분 더 볶아 완성합니다.
  5. 접시에 담고 파김치는 곁들이는 반찬으로 함께 준비합니다.

추천음식 : 계란 장조림
레시피 : 
  1. 계란을 냄비에 넣고 물을 충분히 부어 중불에서 완숙으로 삶습니다.
  2. 삶은 계란은 껍질을 벗겨 준비합니다.
  3. 장조림 국물과 고기를 작은 팬에 넣고 중약불에서 데우다가 삶은 계란을 넣어 함께 조립니다.
  4. 계란이 장조림 국물 맛을 충분히 흡수하도록 약 10분 정도 은근히 졸입니다.
  5. 고기와 계란을 접시에 담아 함께 내면 완성입니다.



## Job Interview 준비

In [11]:
# 채용 공고/스펙을 입력받아, 면접 질문과 모범 답안을 JSON(dict) 형태로 반환하는 함수
def job_interview_json(user_input: str, /, *, model='gpt-4.1-mini', temperature=1, top_p=1, max_completion_tokens=2048):

    response = client.chat.completions.create(    # Chat Completion API 호출
        model = model,
        messages = [
            {
                "role": "system",  # 시스템 프롬프트 : 모델의 페르소나/규칙 설정
                "content": [
                    {
                        "type": "text",  # 텍스트 타입 컨텐츠
                        # 시스템 프롬프트 : 규칙/포맷/예시 포함
                        "text": """
# Instruction
당신은 20년 경력의 ML/DL 엔지니어이고, 이번 신입개발자 채용의 면접관이다.
주어진 job posting(회사정보)를 바탕으로 신입개발자에 제공할 면접 질문과 모범 답변을 작성하세요.
크게 hard skill과 soft skill/leadership 질문을 구분하여 제시하십시오.

아래 지침을 반드시 준수하세요:

- 면접 질문 및 답변은 반드시 한글로, 그리고 신입 개발자에게 현실적으로 맞도록 작성합니다.
- hard skill(기술 질문)과 soft skill/leadership(소통, 태도, 리더십 등) 영역을 구분해 각각 최소 2개 이상의 예시를 만듭니다.
- 입력 정보가 부족한 경우, 통상적인 산업/포지션 상황에 근거해 논리적으로 추론해 주세요.
- 답변 전체의 output format은 반드시 아래 JSON 형태여야 하며, reasoning > hard_skill > soft_skill_leadership 순서로, 각 영역에 반드시 "질문"과 "모범답변"이 쌍으로 들어갑니다.
- 입력값 미제공 시 [회사정보], [스펙] 등의 placeholder를 사용하세요.

# Output Format

아래 JSON 포맷으로 여는 코드블럭 없이 반드시 출력합니다:

{
  "hard_skill": [
    {
      "question": "[hard skill(기술 중점) 면접 질문]",
      "answer": "[신입개발자 입장에서 모범답변]"
    },
    …
  ],
  "soft_skill_leadership": [
    {
      "question": "[soft skill/leadership(태도, 커뮤니케이션, 성장잠재력 등) 면접 질문]",
      "answer": "[신입개발자 입장에서 모범답변]"
    },
    …
  ]
}

- 각 영역별 면접 질문과 모범답변은 모두 한글로 작성합니다.
- 반드시 하드 스킬, 소프트 스킬/리더십 항목을 구분하여 각 2개 이상 작성합니다(총 4개 이상).
- 예시(Example)처럼 구조, 문장 길이, 구체성, 답변 스타일을 맞추되, 지원자는 신입개발자임을 꼭 반영하세요.

# 예시(Example)

Input:
회사정보: [AI 기업, 제조공정 데이터 분석 솔루션 개발]
스펙: [전자공학 전공, 신입 개발자, Python/Java 가능, 인턴 경험 있음]

Output:

{{
  "hard_skill": [
    {
      "question": "Python 언어를 사용하여 데이터 전처리 경험이 있나요? 예시를 들어 설명해 주세요.",
      "answer": "대학 시절 프로젝트에서 Pandas를 이용해 결측치 처리와 이상치 제거를 해본 경험이 있습니다. 데이터 정제의 중요성을 실제로 체감할 수 있었습니다."
    },
    {
      "question": "제조 데이터와 같이 구조적인 데이터를 다룰 때 주의해야 하는 점은 무엇이라고 생각하나요?",
      "answer": "데이터의 정확한 구조 파악과, 이상치 및 에러를 사전에 점검하는 것이 중요하다고 생각합니다."
    }
  ],
  "soft_skill_leadership": [
    {
      "question": "팀 프로젝트에서 갈등이 있을 때 어떻게 해결해보셨나요?",
      "answer": "인턴 경험 중 팀원과 의견이 달랐을 때 서로의 입장을 경청한 뒤 중간 지점을 찾아 협력한 경험이 있습니다."
    },
    {
      "question": "빠르게 변화하는 환경에서 새로운 기술을 습득했던 경험이 있나요?",
      "answer": "새로운 도구가 필요했던 프로젝트에서 적극적으로 온라인 자료를 찾아 공부하며 습득하였습니다."
    }
  ]
}}

# Notes
- 지원자는 신입개발자, 면접관 페르소나는 20년차 ml/dl 엔지니어임을 모든 답변에 일관되게 반영하세요.
- 모든 콘텍스트와 답변, 지시문, reasoning, 사례, placeholder, Q&A 등은 전체 한글로 작성합니다.
- 포맷(최상단 하드스킬 > 소프트스킬/리더십)과 다국어 혼용 불허, 신입 시점 미반영 등은 모두 미승인 처리 대상입니다.

**Objective Reminder:**
1. '20년차 ml/dl engineer' 페르소나로서, 신입 개발자 대상 면접 Q&A를 회사/직무 정보로부터 reasoning → hard/soft skill/leaderhip별 제시
2. JSON 아웃풋 포맷만 사용 (코드블럭 넣지 않음)
"""
                    }
                ]
            },
            {
                "role": "user",    # 유저 프롬프트 : 실제 입력 데이터
                "content": [
                    {
                        "type": "text",
                        "text": user_input
                    }
                ]
            }
        ],
        response_format = {"type": "json_object"},      # 응답 형식을 JSON 객체로 강제
        temperature = temperature,                      # 모델의 랜덤성/다양성 (낮으면 일관성, 높으면 창의적)
        max_completion_tokens = max_completion_tokens,  # 최대 출력 토큰
        top_p = top_p,                                  # 누적확률 p까지의 후보를 샘플링 (1이면 전체사용)
        frequency_penalty = 0,               # 동일 단어/구절 반복시 감점 (반복 억제)
        presence_penalty = 0,                # 이미 등장한 주제/단어 사용시 감점 (반복 억제)
        store = False                        # 응답을 서버에 저장/로깅 할지 설정 (False면 저장 안함)
    )

    return json.loads(response.choices[0].message.content)  # JSON 문자열 -> 파이썬 dict로 파싱

In [14]:
# 입력된 공고 내용을 기반으로 면접 질문/모범답안 JSON 생성
output = job_interview_json("""
## Job Descriptions

주요 업무 내용 안내

### 코딕스 개발팀 소개말

코딕스 개발팀은 콘텐츠 플랫폼 테스트북을 중심으로 일하고 있습니다. 사용자들이 더 편하게 서비스를 이용하도록 돕고있어요. 테스트북 뿐만 아니라 웹을 기반으로 한 다양한 서비스를 개발합니다.
기존에 사용하던 도구 이외에 새로운 Framework나 기술에도 관심이 많으며 적극적으로 검토하고 도입하기 위해 노력해요. 물론 개인의 역량과 커리어 증진에도 힘쓰고 있습니다.

---

## AI 개발자 주요 업무 내용 안내

### AI 개발자 (AI Developer)

AI 기술을 이용한 교육용 애플리케이션 프로젝트를 개발(ML/DL/Generative AI)하고,
Python 기반 API 서버 개발 및 Kubernetes 기반 배포 환경 구성합니다.

---

### 기술 및 자격 요건

* Python 프로그래밍 (FastAPI, Django, Flask 등) 경험이 있는 사람
* NLP 프로젝트 또는 LLM + RAG + VectorDB 기반 개발 경험이 있는 사람
* RESTful API 설계 및 Kubernetes 기반 배포 경험이 있는 사람
* 업무 커뮤니케이션에 대한 소통 능력이 뛰어난 분

---

### 우대 사항

* 바이브 코딩 경험을 보유하신 분
* 생성형 AI 관련 프로젝트 경험이 있는 사람
* 각종 협업 도구에 익숙하고 새로운 사용에 적극적인 사람
""")

In [16]:
output

{'hard_skill': [{'question': 'Python을 사용해서 FastAPI나 Django 같은 프레임워크로 간단한 RESTful API를 만들어 본 경험이 있나요? 구체적으로 어떤 기능을 구현했는지 설명해 주세요.',
   'answer': '대학 프로젝트에서 FastAPI를 사용해 회원가입과 로그인 기능을 제공하는 간단한 RESTful API를 만들어 본 경험이 있습니다. 사용자의 요청에 따라 데이터를 처리하고 JSON 형식으로 응답하는 기본적인 구조를 이해하게 되었습니다.'},
  {'question': 'NLP 또는 LLM(대형 언어 모델) 관련 프로젝트 경험이 있다고 하셨는데, 경험한 프로젝트에서 어떤 자연어 처리 기술이나 도구를 사용하였는지 구체적으로 설명해 주세요.',
   'answer': '캡스톤 프로젝트에서 Hugging Face의 Transformers 라이브러리를 활용해 문서 요약이나 감성 분석 모델을 구현했습니다. 전처리 과정에서는 토큰화와 불용어 제거를 수행했고, 파인튜닝으로 모델 정확도를 높이는 실습을 했습니다.'}],
 'soft_skill_leadership': [{'question': '개발팀 내에서 새로운 기술 도입 여부를 논의할 때, 본인의 의견을 어떻게 전달하고 팀원들과 조율하는 편인가요?',
   'answer': '저는 먼저 새로운 기술의 장단점과 도입 시 예상되는 효과를 정리해서 명확하게 설명하려고 노력합니다. 그리고 팀원들의 의견을 경청하며, 궁금한 점이나 우려가 있을 때는 직접 질문하고 함께 해결책을 찾아가는 방식으로 소통합니다.'},
  {'question': '협업 도구나 새로운 기술을 배워야 할 때 어떻게 접근하시나요? 그리고 그 경험이 본인에게 어떤 도움이 되었나요?',
   'answer': '처음 접하는 도구나 기술은 공식 문서와 튜토리얼부터 차근차근 학습합니다. 필요 시에는 동료나 인터넷 커뮤니티에 질문하며 빠르게 습득하려고 노력합니다. 이 과정에서 자기 주도적 학습 능력과 문제 해결 능

In [30]:
for category, qa_list in output.items():    # output(dict)의 key:value 형태로 뽑아내서 순회
    print(f'[{category}]')

    for qa in qa_list:                      # Q&A 리스트를 순회
        question = qa.get("question", "").strip()  # qestion 키 값 추출(없으면 빈 문자열)
        answer = qa.get("answer", "").strip()      # answer 키 값 추출(없으면 빈 문자열)

        print(f"Q) {question}")
        print(f"A) {answer}")
        
    print()

[hard_skill]
Q) Python을 사용해서 FastAPI나 Django 같은 프레임워크로 간단한 RESTful API를 만들어 본 경험이 있나요? 구체적으로 어떤 기능을 구현했는지 설명해 주세요.
A) 대학 프로젝트에서 FastAPI를 사용해 회원가입과 로그인 기능을 제공하는 간단한 RESTful API를 만들어 본 경험이 있습니다. 사용자의 요청에 따라 데이터를 처리하고 JSON 형식으로 응답하는 기본적인 구조를 이해하게 되었습니다.
Q) NLP 또는 LLM(대형 언어 모델) 관련 프로젝트 경험이 있다고 하셨는데, 경험한 프로젝트에서 어떤 자연어 처리 기술이나 도구를 사용하였는지 구체적으로 설명해 주세요.
A) 캡스톤 프로젝트에서 Hugging Face의 Transformers 라이브러리를 활용해 문서 요약이나 감성 분석 모델을 구현했습니다. 전처리 과정에서는 토큰화와 불용어 제거를 수행했고, 파인튜닝으로 모델 정확도를 높이는 실습을 했습니다.

[soft_skill_leadership]
Q) 개발팀 내에서 새로운 기술 도입 여부를 논의할 때, 본인의 의견을 어떻게 전달하고 팀원들과 조율하는 편인가요?
A) 저는 먼저 새로운 기술의 장단점과 도입 시 예상되는 효과를 정리해서 명확하게 설명하려고 노력합니다. 그리고 팀원들의 의견을 경청하며, 궁금한 점이나 우려가 있을 때는 직접 질문하고 함께 해결책을 찾아가는 방식으로 소통합니다.
Q) 협업 도구나 새로운 기술을 배워야 할 때 어떻게 접근하시나요? 그리고 그 경험이 본인에게 어떤 도움이 되었나요?
A) 처음 접하는 도구나 기술은 공식 문서와 튜토리얼부터 차근차근 학습합니다. 필요 시에는 동료나 인터넷 커뮤니티에 질문하며 빠르게 습득하려고 노력합니다. 이 과정에서 자기 주도적 학습 능력과 문제 해결 능력이 향상되어, 팀 업무 효율에도 긍정적인 영향을 끼친 경험이 있습니다.



In [13]:
job_interview_json("""
## ENSSEL

### AI 사업부

### AI(인공지능) 개발자

---

## 모집 분야

### 모집 부문

AI 사업부
AI(인공지능) 개발자

### 업무 내용

* 인공지능 알고리즘 개발 및 연구
* 인공지능 서비스 솔루션 개발

  * Vision
  * Time Series
  * NLP
* 데이터 처리 및 분석
* 언어 : Python

### 자격 요건 및 우대사항

* 대졸 이상
* 전반적인 AI 기술에 대한 이해가 있으신 분

  * Vision / TimeSeries / NLP
  * 딥러닝 모델에 대한 이해 및 학습 경험
* 딥러닝 프레임워크 (Pytorch, Tensorflow 등) 사용
* 데이터를 분석하거나 모델링을 통해 결과를 도출해본 경험이 있으신 분
* *경력자의 경우, PMP 자격증 소지자 우대

### 모집 인원

0명
""")

{'hard_skill': [{'question': 'Python을 사용하여 딥러닝 모델을 구현해 본 경험이 있나요? 구체적인 프로젝트나 학습 경험을 말씀해 주세요.',
   'answer': '대학 시절에 CNN을 이용한 이미지 분류 프로젝트를 진행한 경험이 있습니다. Python과 PyTorch를 사용해서 데이터 전처리부터 모델 설계, 학습, 평가까지 전 과정을 경험했습니다.'},
  {'question': 'Vision, Time Series, NLP 중 관심 있는 분야가 있나요? 해당 분야에서 적용 가능한 딥러닝 모델이나 알고리즘에 대해 설명해 주세요.',
   'answer': '저는 NLP 분야에 관심이 많아 자연어 처리 프로젝트를 진행했습니다. 기본적인 RNN과 Transformer 모델을 활용해 텍스트 분류 및 감성 분석을 수행해 봤습니다.'}],
 'soft_skill_leadership': [{'question': '팀으로 AI 프로젝트를 수행할 때 맡은 역할이 무엇이었고, 협업 과정에서 중요하게 생각한 점은 무엇인가요?',
   'answer': '팀 프로젝트에서 데이터 전처리와 모델 학습 파트를 맡았습니다. 서로 의견을 경청하고, 소통을 통해 문제점을 빠르게 공유하는 것이 협업에 중요하다고 생각했습니다.'},
  {'question': 'AI 기술은 빠르게 발전하고 있습니다. 신입 개발자로서 새로운 기술을 어떻게 학습하고 따라가고 있나요?',
   'answer': '온라인 강의, 논문, 오픈소스 코드를 꾸준히 학습하며 최신 동향을 파악하고 있습니다. 특히, 실습 위주로 공부해 직접 모델을 구현하는 것을 중요하게 생각합니다.'}]}

In [31]:
for category, qa_list in output.items():    # output(dict)의 key:value 형태로 뽑아내서 순회
    print(f'[{category}]')

    for qa in qa_list:                      # Q&A 리스트를 순회
        question = qa.get("question", "").strip()  # qestion 키 값 추출(없으면 빈 문자열)
        answer = qa.get("answer", "").strip()      # answer 키 값 추출(없으면 빈 문자열)

        print(f"Q) {question}")
        print(f"A) {answer}")
        
    print()

[hard_skill]
Q) Python을 사용해서 FastAPI나 Django 같은 프레임워크로 간단한 RESTful API를 만들어 본 경험이 있나요? 구체적으로 어떤 기능을 구현했는지 설명해 주세요.
A) 대학 프로젝트에서 FastAPI를 사용해 회원가입과 로그인 기능을 제공하는 간단한 RESTful API를 만들어 본 경험이 있습니다. 사용자의 요청에 따라 데이터를 처리하고 JSON 형식으로 응답하는 기본적인 구조를 이해하게 되었습니다.
Q) NLP 또는 LLM(대형 언어 모델) 관련 프로젝트 경험이 있다고 하셨는데, 경험한 프로젝트에서 어떤 자연어 처리 기술이나 도구를 사용하였는지 구체적으로 설명해 주세요.
A) 캡스톤 프로젝트에서 Hugging Face의 Transformers 라이브러리를 활용해 문서 요약이나 감성 분석 모델을 구현했습니다. 전처리 과정에서는 토큰화와 불용어 제거를 수행했고, 파인튜닝으로 모델 정확도를 높이는 실습을 했습니다.

[soft_skill_leadership]
Q) 개발팀 내에서 새로운 기술 도입 여부를 논의할 때, 본인의 의견을 어떻게 전달하고 팀원들과 조율하는 편인가요?
A) 저는 먼저 새로운 기술의 장단점과 도입 시 예상되는 효과를 정리해서 명확하게 설명하려고 노력합니다. 그리고 팀원들의 의견을 경청하며, 궁금한 점이나 우려가 있을 때는 직접 질문하고 함께 해결책을 찾아가는 방식으로 소통합니다.
Q) 협업 도구나 새로운 기술을 배워야 할 때 어떻게 접근하시나요? 그리고 그 경험이 본인에게 어떤 도움이 되었나요?
A) 처음 접하는 도구나 기술은 공식 문서와 튜토리얼부터 차근차근 학습합니다. 필요 시에는 동료나 인터넷 커뮤니티에 질문하며 빠르게 습득하려고 노력합니다. 이 과정에서 자기 주도적 학습 능력과 문제 해결 능력이 향상되어, 팀 업무 효율에도 긍정적인 영향을 끼친 경험이 있습니다.

